# Imports

In [1]:
# from kfp import dsl
# from kfp.v2 import compiler
from google.cloud import aiplatform
from typing import List, Dict
import boto3
from botocore.exceptions import ClientError
import json, os, ast, re
from datetime import datetime, timedelta
import pandas as pd, numpy as np
from scipy.special import softmax
from pydantic import BaseModel, Field, ValidationError

import scrubadub, scrubadub_spacy

import snowflake.connector as sc
from snowflake.connector.pandas_tools import write_pandas

import vertexai
import vertexai.preview.generative_models as generative_models
from vertexai.generative_models import GenerativeModel, GenerationConfig, Part

# Sentiments
from transformers import pipeline
from transformers import AutoTokenizer, AutoConfig
from transformers import AutoModelForSequenceClassification

/Users/sunsin/Installs/anaconda3/envs/posigen/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Variables

In [2]:
# Temporary secrets manager
with open("secrets.json", 'r') as secrets_file:
    secrets = json.load(secrets_file)
aws_access_key = secrets.get("aws_access_key")
aws_secret_key = secrets.get("aws_secret_key")

# AWS
s3_source_bucket = secrets.get('s3_source_bucket')
s3_transcripts_location = secrets.get('s3_transcripts_location')

# GCP
gcp_project_id=secrets.get('gcp_project_id')
gcp_prjct_location=secrets.get('gcp_prjct_location')

# Snowflake
private_key_file = secrets.get('snowflakegcp_rsa_key')
private_key_file_pwd = secrets.get('snf_ssh_key_pass')

conn_params = {
    'account': secrets.get('snf_account'),
    'user': secrets.get('snf_user'),
    'private_key_file': secrets.get('snf_private_key_file'),
    'private_key_file_pwd':secrets.get('snf_private_key_pwd'),
    'warehouse': secrets.get('snf_warehouse'),
    'database': secrets.get('snf_database'),
    'schema': secrets.get('snf_schema')
}

# # Sentiment Scores
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model_sentiment = AutoModelForSequenceClassification.from_pretrained(MODEL)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Util Functions

## Misc Utils

### Initiate Master Inter and Intra Dataframes

In [3]:
def initiate_master_dataframes():
    if os.path.isfile("df_intra_calls_data.csv"):
        print("df_intra_calls_data.csv exists.") 
        df_intra_calls_data = pd.read_csv("df_intra_calls_data.csv")
        df_intra_calls_data.CONTACT_ID = df_intra_calls_data.CONTACT_ID.astype('string')
    else:
        print("df_intra_calls_data.csv does not exists.")
        df_intra_calls_data = pd.DataFrame()

    if os.path.isfile("df_inter_calls_data.csv"):
        print("df_inter_calls_data.csv exists.")
        df_inter_calls_data = pd.read_csv("df_inter_calls_data.csv")
        df_inter_calls_data.CONTACT_ID = df_inter_calls_data.CONTACT_ID.astype('string')
    else:
        print("df_inter_calls_data.csv does not exists.")
        df_inter_calls_data = pd.DataFrame()

    return df_intra_calls_data, df_inter_calls_data

## Function: Listing Transcripts

In [4]:
def list_new_transcripts(aws_access_key: str, aws_secret_key: str, source_bucket: str, custom_location: str, max_objects: int):
    """
    Fetch audio file from S3 and return it as a BytesIO object
    """
    try:
        s3_client = boto3.client(
            's3',
            aws_access_key_id=aws_access_key,
            aws_secret_access_key=aws_secret_key
        )

        print()
        print("New Transcripts to process")
        # List files in the folder
        response = s3_client.list_objects_v2(Bucket=s3_source_bucket, Prefix=s3_transcripts_location)
        
        list_transcripts = []
        for obj in response.get('Contents', []):
            if obj['Key'].endswith('.json'):
                list_transcripts.append([obj['Key'], obj['LastModified']])
                print(f"{str(obj['LastModified']) +": "+ obj['Key']}")
                if len(list_transcripts) >= max_objects:
                    break  # Exit the loop after printing max objects
    
        return list_transcripts

    except ClientError as e:
        print(f"Error accessing S3: {e}")
        raise

## Function: Read Transcripts

In [5]:
def read_new_transcripts(aws_access_key: str, aws_secret_key: str, s3_source_bucket: str, file_key):
    """
    Read Transcript JSON content from a specific file in S3.
    
    :param bucket_name: Name of the S3 bucket
    :param file_key: Full path/key of the JSON file
    :return: Parsed JSON content
    """
    s3_client = boto3.client(
            's3',
            aws_access_key_id=aws_access_key,
            aws_secret_access_key=aws_secret_key
        )
    
    try:
        # Download the file
        response = s3_client.get_object(Bucket=s3_source_bucket, Key=file_key)
        
        # Read the content
        json_content = response['Body'].read().decode('utf-8')
        
        # Parse JSON
        return json.loads(json_content)
    
    except Exception as e:
        print(f"Error reading Transcript JSON file {file_key}: {e}")
        print()
        return None


# Create Intra-call Dataframe

In [6]:
def millis_to_hhmmss(millis):
    """Convert milliseconds to mm:ss format"""
    total_seconds = int(millis / 1000)
    hours = total_seconds // 3600
    minutes = total_seconds // 60
    seconds = total_seconds % 60
    return f"{minutes:02d}:{seconds:02d}"

def convert_to_seconds(time_str):
    try:
        # Parse time string using datetime
        time_obj = datetime.strptime(time_str, '%H:%M:%S')
        # Convert to timedelta and extract total seconds
        total_seconds = time_obj.minute * 60 + time_obj.second
        return total_seconds
    except ValueError:
        return None

def process_transcript(
    transcript_data: dict,
    contact_id: str
):
    """
    Pre-process the transcript loaded from S3 Buckets:
    1. Load the transcript as Pandas Dataframe.
    2. Select only the necessary columns ['BeginOffsetMillis', 'EndOffsetMillis', 'ParticipantId', 'Content', 'Sentiment', 'LoudnessScore'].
    3. Format the time in minutes and seconds.
    4. Rename the columns for better understanding.
    """
    # Load the Transcript as Pandas Dataframe
    transcript_df = pd.json_normalize(transcript_data['Transcript'])

    # Select the relevant Columns
    columns_to_select = [
        'BeginOffsetMillis',
        'EndOffsetMillis',
        'ParticipantId',
        'Content'
    ]
    formatted_df = transcript_df[columns_to_select].copy()
    
    # Optionally rename columns to reflect their new format
    formatted_df = formatted_df.rename(columns={
        'BeginOffsetMillis': 'Begin_Offset',
        'EndOffsetMillis': 'End_Offset',
        'Content': 'caption',
        'Sentiment': 'sentiment_label',
        'ParticipantId': 'speaker_tag'
    })

    # Inserting the Call ID:
    formatted_df.insert(loc=0, column='contact_id', value=contact_id)
    formatted_df['call_language'] = transcript_data['LanguageCode']

    return formatted_df

def get_sentiment_label(row):
    # Check conditions in order of priority (Positive > Negative > Neutral)
    if row['positive'] > row['negative'] and row['positive'] > row['neutral']:
        return 'Positive'
    elif row['negative'] > row['positive'] and row['negative'] > row['neutral']:
        return 'Negative'
    else:
        return 'Neutral'

def get_sentiment_scores(text_list):
    dict_sentiments = []
    for text in text_list:
        encoded_input = tokenizer(text, return_tensors='pt')
        output = model_sentiment(**encoded_input)
        scores = output[0][0].detach().numpy()
        scores = np.round(np.multiply(softmax(scores), 100), 2)
        merged_dict = dict(zip(list(config.id2label.values()), list(scores)))
        dict_sentiments.append(merged_dict)

    df_dict_sentiments = pd.DataFrame(dict_sentiments)
    df_dict_sentiments['sentiment_lable'] = df_dict_sentiments[['positive','negative','neutral']].apply(get_sentiment_label, axis=1)
    
    return df_dict_sentiments

def get_different_times(intra_call):
    # Apply formatting to both time columns
    intra_call['start_time_second'] = (intra_call['Begin_Offset'] / 1000).astype(int)
    # intra_call['Begin_Offset'] = intra_call['Begin_Offset'].apply(millis_to_hhmmss)
    intra_call['end_time_second'] = (intra_call['End_Offset'] / 1000).astype(int)
    # intra_call['End_Offset'] = intra_call['End_Offset'].apply(millis_to_hhmmss)
    intra_call['time_spoken_second'] = intra_call['end_time_second'] - intra_call['start_time_second']
    intra_call['time_spoken_second'] = intra_call['time_spoken_second'].where(intra_call['time_spoken_second'] >= 0, 0)
    intra_call['time_spoken_second'] = intra_call['time_spoken_second'].fillna(0).astype(int)
    intra_call['time_silence_second'] = intra_call['start_time_second'].shift(-1) - intra_call['end_time_second']
    intra_call['time_silence_second'] = intra_call['time_silence_second'].where(intra_call['time_silence_second'] >= 0, 0)
    intra_call['time_silence_second'] = intra_call['time_silence_second'].fillna(0).astype(int)
    intra_call['load_date'] = datetime.now()

    # Dropping time formatted columns
    intra_call = intra_call.drop(['Begin_Offset', 'End_Offset'], axis=1)

    return intra_call
    
def create_intra_call_df(aws_access_key: str, aws_secret_key: str, transcript_data: dict, contact_id: str):
    print(f"Creating Intra Call df")
        
    # Get the relevant columns from the loaded transcript file
    intra_call = process_transcript(transcript_data, contact_id)
    
    df_sentiment_scores = get_sentiment_scores(intra_call.caption.to_list())
    intra_call = pd.concat([intra_call, df_sentiment_scores], axis=1)

    intra_call = get_different_times(intra_call)
    print(f"Completed Intra Call df")
    print()

    return intra_call

# Create Inter-call Dataframe

In [7]:
# Pydantic models for validation
class CallSummary(BaseModel):
    summary: str = Field(..., max_length=500)
    key_points: List[str] = Field(..., max_items=5)
    outcome: str = Field(..., max_length=200)
    follow_up_recommendations: List[str] = Field(..., max_items=3)

class CallTopic(BaseModel):
    primary_topic: str = Field(..., max_length=100)
    category: str = Field(..., max_length=100)
    sub_category: str = Field(..., max_length=100)

class AgentCoaching(BaseModel):
    strengths: List[str] = Field(..., max_items=3)
    improvement_areas: List[str] = Field(..., max_items=3)
    specific_recommendations: List[str] = Field(..., max_items=4)
    skill_development_focus: List[str] = Field(..., max_items=3)

class TranscriptAnalysis(BaseModel):
    call_summary: CallSummary
    call_topic: CallTopic
    agent_coaching: AgentCoaching

class KPIExtractor:
    def __init__(self, project_id: str, location: str):
        vertexai.init(project=project_id, location=location)
        self.model = GenerativeModel("gemini-1.5-flash-002")
        self.generation_config = {
            "temperature": 0.3,
            "max_output_tokens": 1024,
            "top_p": 0.8,
            "top_k": 40,
            "response_format": "json"
        }
        self.safety_settings = {
            generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
            generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
            generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
            generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        }
        
    def create_prompt(self, transcript: str) -> str:
        """Create a structured prompt for KPI extraction"""
        return f"""
        Analyze this call transcript and provide a structured analysis in the exact JSON format specified below.
        Keep responses concise, specific, and actionable.

        Guidelines:
        - Call summary should be factual and highlight key interactions
        - Topics and categories should match standard business taxonomies
        - Coaching points should be specific and actionable
        - All responses must follow the exact structure specified
        - Ensure all lists have the specified maximum number of items
        - All text fields must be clear, professional, and free of fluff

        Transcript:
        {transcript}

        Required Output Structure:
        {{
            "call_summary": {{
                "summary": "3-4 line overview of the call",
                "key_points": ["Point 1", "Point 2", "Point 3", "Point 4", "Point 5"],
                "outcome": "Clear statement of call resolution",
                "follow_up_recommendations": ["Rec 1", "Rec 2", "Rec 3"]
            }},
            "call_topic": {{
                "primary_topic": "Main topic of discussion",
                "category": "Business category",
                "sub_category": "Specific sub-category"
            }},
            "agent_coaching": {{
                "strengths": ["Strength 1", "Strength 2", "Strength 3"],
                "improvement_areas": ["Area 1", "Area 2", "Area 3"],
                "specific_recommendations": ["Rec 1", "Rec 2", "Rec 3", "Rec 4"],
                "skill_development_focus": ["Skill 1", "Skill 2", "Skill 3"]
            }}
        }}

        Rules:
        1. Maintain exact JSON structure
        2. No additional fields or comments
        3. No markdown formatting
        4. Ensure all arrays have the exact number of items specified
        5. Keep all text concise and professional
        6. Do not mention any PII information such as Customer Name etc.
        """
    
    
    def extract_json(self, response: str):
        """Extracts valid JSON from a response that may contain extra characters like ```json."""
        match = re.search(r'```json\s*([\s\S]*?)\s*```', response)
        if match:
            json_str = match.group(1)  # Extract JSON content
        else:
            json_str = response.strip()  # If no markdown, assume raw JSON
        
        try:
            return json.loads(json_str)  # Convert to dictionary
        except json.JSONDecodeError:
            raise ValueError("Invalid JSON response")
    
            
    def validate_response(self, response_json: Dict) -> TranscriptAnalysis:
        """Validate the response using Pydantic models"""
        try:
            return TranscriptAnalysis(**response_json)
        except ValidationError as e:
            print(f"Skipping call {i + 1}: Error extracting KPIs - {e}")

    def extract_genai_kpis(self, transcript: str):
        """
        Extract KPIs from transcript using Gemini API
        
        Args:
            transcript (str): Call transcript text
            
        Returns:
            Dict: Structured KPI data or None if extraction fails
        """
        try:
            # Generate prompt
            prompt = self.create_prompt(transcript)
            
            # Get response from Gemini
            response = self.model.generate_content(prompt)
            
            # Parse JSON response
            response_json = self.extract_json(response.text)
            
            # Validate response structure
            validated_response = self.validate_response(response_json)
            
            return validated_response.model_dump()
            
        except Exception as e:
            print(f"Error extracting KPIs: {str(e)}")
            return None

def dict_to_newline_string(data: dict) -> str:
    """Converts a dictionary into a new-line formatted string."""
    formatted_str = ""
    for key, value in data.items():
        formatted_str += f"{key}:\n"
        for item in value:
            formatted_str += f"  - {item}\n"
    return formatted_str.strip()
    
def create_inter_call_df(
    gcp_project_id: str,
    gcp_prjct_location: str,
    df_intra_call: pd.DataFrame,
    transcript_data: dict,
    ac_last_modified_date: datetime
):
    # Redact PII Data
    print("Redacting PII data.")
    scrubber = scrubadub.Scrubber()
    scrubber.add_detector(scrubadub_spacy.detectors.SpacyEntityDetector)
    df_intra_call.caption = df_intra_call.caption.apply(scrubber.clean)

    # Extract KPIs from Gemini
    extractor = KPIExtractor(gcp_project_id, gcp_prjct_location)
    transcript = " ".join(df_intra_call.caption)
    call_gen_kpis = extractor.extract_genai_kpis(transcript)

    print(f"Creating Inter Call df")
    inter_call_dict = {}
    inter_call_dict['contact_id'] = str(df_intra_call['contact_id'][0])
    inter_call_dict['call_text'] = " ".join(df_intra_call.caption)
    inter_call_dict['call_summary'] = call_gen_kpis['call_summary']['summary']
    inter_call_dict['topic'] = call_gen_kpis['call_topic']['primary_topic']
    inter_call_dict['category'] = "Static Category TBD"
    inter_call_dict['category_generated'] = call_gen_kpis['call_topic']['category']
    inter_call_dict['sub_category'] = "Static Sub-Category TBD"
    inter_call_dict['sub_category_generated'] = call_gen_kpis['call_topic']['sub_category']
    inter_call_dict['agent_coaching'] = dict_to_newline_string(call_gen_kpis['agent_coaching'])

    df_inter_call = pd.DataFrame(pd.Series(inter_call_dict)).T

    # Add metadata from AWS
    # df_inter_call['account_id'] = transcript_data['AccountId']
    df_inter_call['agent_speech_speed'] = transcript_data['ConversationCharacteristics']['TalkSpeed']['DetailsByParticipant']['AGENT']['AverageWordsPerMinute']
    df_inter_call['customer_speech_speed'] = transcript_data['ConversationCharacteristics']['TalkSpeed']['DetailsByParticipant']['CUSTOMER']['AverageWordsPerMinute']
    df_inter_call['total_talktime_agent_second'] = transcript_data['ConversationCharacteristics']['TalkTime']['DetailsByParticipant']['AGENT']['TotalTimeMillis']
    df_inter_call['total_talktime_customer_second'] = transcript_data['ConversationCharacteristics']['TalkTime']['DetailsByParticipant']['CUSTOMER']['TotalTimeMillis']
    df_inter_call['total_talktime_call_second'] = transcript_data['ConversationCharacteristics']['TalkTime']['TotalTimeMillis']
    df_inter_call['total_duration_call_second'] = transcript_data['ConversationCharacteristics']['TotalConversationDurationMillis']
    df_inter_call['total_dead_air_call_second'] = df_inter_call['total_duration_call_second'] - df_inter_call['total_talktime_call_second']
    # df_inter_call['customer_instance_id'] = transcript_data['CustomerMetadata']['InstanceId']
    # df_inter_call['call_job_status'] = transcript_data['JobStatus']
    df_inter_call['call_language'] = transcript_data['LanguageCode']
    df_inter_call['call_s3_uri'] = transcript_data['CustomerMetadata']['InputS3Uri']
    df_inter_call['ac_last_modified_date'] = ac_last_modified_date
    df_inter_call['load_date'] = datetime.now()
    
    print(f"Completed Inter Call df")
    print()
    
    return df_inter_call

# Writing Dataframe to Snowflake

In [8]:
def insert_new_records(conn, table_name, df):
    """
    Inserts only new records (based on ID) into Snowflake table.
    1. Fetches existing IDs from table
    2. Filters out rows with existing IDs from DataFrame
    3. Inserts only new records
    
    Args:
        conn: Snowflake connection object
        table_name: Name of the target table
        df: Pandas DataFrame containing the data (must have 'id' column)
    """
    cursor = conn.cursor()
    
    # Get existing IDs from Snowflake table
    cursor.execute(f"SELECT DISTINCT(CONTACT_ID) FROM {table_name}")
    existing_ids = {row[0] for row in cursor.fetchall()}
    
    # Filter DataFrame to keep only new records
    new_records_df = df[~df['CONTACT_ID'].isin(existing_ids)]
    
    if len(new_records_df) == 0:
        print("No new records to insert")
        return 0
    
    # Insert new records
    success, nchunks, nrows, _ = write_pandas(conn, new_records_df, table_name)
    
    print(f"Inserted {nrows} new records")
    print(f"Skipped {len(df) - len(new_records_df)} existing records")
    
    cursor.close()
    return nrows

# Main Function

In [9]:
# Initiating Master DataFrames
df_intra_calls_data, df_inter_calls_data = initiate_master_dataframes()

# Get the transcripts in to_process_folder
max_objects = 20
list_transcripts = list_new_transcripts(aws_access_key, aws_secret_key, s3_source_bucket, s3_transcripts_location, max_objects)

# If there are transcripts to be processed
if len(list_transcripts) == 0:
    print("No Transcripts to Process")
    print()

else:
    print()
    print(f"Transcripts to process: {len(list_transcripts)}")
    print()
    # Process the call
    for transcript in list_transcripts:
        # get the call ID
        contact_id = transcript[0].split('/')[-1].split('.')[0].split('analysis')[0].strip('_')
        ac_last_modified_date = datetime.strptime(transcript[0].split('analysis_')[-1].split('.')[0].replace('_', ':'), '%Y-%m-%dT%H:%M:%SZ')
        print()
        print()
        print(f"Processing Call: {contact_id}")        

        # Check if Call Already Processed
        if (len(df_intra_calls_data) > 0 and contact_id in df_intra_calls_data.CONTACT_ID.unique()) and (len(df_inter_calls_data) > 0 and contact_id in df_inter_calls_data.CONTACT_ID.unique()):
            print("Call already Processed.")
            print()
            # break

        else:
            # get the audio transcript file name
            transcript_file = transcript[0]
            
            # Get the Transcript file from S3 Bucket
            transcript_data = read_new_transcripts(aws_access_key, aws_secret_key, s3_source_bucket, transcript_file)
            print("Successfully loaded the Transcript JSON")
            print()
        
            # Create the Inter Call KPIs
            df_intra_call = create_intra_call_df(aws_access_key, aws_secret_key, transcript_data, contact_id)
        
            # Create the Intra Call KPIs
            df_inter_call = create_inter_call_df(gcp_project_id, gcp_prjct_location, df_intra_call, transcript_data, ac_last_modified_date)

            ###============================================================###
            # Appending to Intra-calls Master DataFrame
            df_intra_call.columns = df_intra_call.columns.str.upper()  # Capitalising Column names for Snowflake
            df_intra_calls_data = pd.concat([df_intra_calls_data, df_intra_call], ignore_index=True)
            df_intra_calls_data.to_csv("df_intra_calls_data.csv", index=False)
            print("Completed Persisting df_intra_calls_data with "+str(contact_id)+" to CSV.")
            
            # Appending to Inter-calls Master DataFrame
            df_inter_call.columns = df_inter_call.columns.str.upper()  # Capitalising Column names for Snowflake
            df_inter_calls_data = pd.concat([df_inter_calls_data, df_inter_call], ignore_index=True)
            df_inter_calls_data.to_csv("df_inter_calls_data.csv", index=False)
            print("Completed Persisting df_inter_calls_data Dataframe to CSV.")

    print(f"Writing Dataframe to Snowflake.")
    conn = sc.connect(**conn_params)
    table_name ='SRC_GCP_INTER_CALLS'
    insert_new_records(conn, table_name, df_inter_calls_data)
    
    table_name ='SRC_GCP_INTRA_CALLS'
    success, nchunks, nrows, _ = write_pandas(conn, df_intra_calls_data, 'SRC_GCP_INTRA_CALLS', auto_create_table=True)
    conn.close()

    print(f"Completed processing {len(list_transcripts)} Calls")

df_intra_calls_data.csv exists.
df_inter_calls_data.csv exists.

New Transcripts to process
2025-01-30 09:05:04+00:00: connect-audio-files/to_process_transcripts/309a0db8-2735-4537-988a-a66bff37c159_analysis_2025-01-17T21_06_34Z.json
2025-01-30 09:05:03+00:00: connect-audio-files/to_process_transcripts/3c2ddea2-d6e3-4acd-8abd-6eb98c192bd8_analysis_2025-01-06T18_29_29Z.json
2025-01-30 09:05:02+00:00: connect-audio-files/to_process_transcripts/3e747be8-b57e-4e90-954d-dabc1e9a623a_analysis_2025-01-14T19_30_16Z.json
2025-01-30 09:04:52+00:00: connect-audio-files/to_process_transcripts/4a0a35d3-6221-4ccd-be93-83938c24a544_analysis_2025-01-17T19_04_29Z.json
2025-01-30 09:05:05+00:00: connect-audio-files/to_process_transcripts/4abe04de-5d7d-4eb9-a999-99d1c6ef54c1_analysis_2024-12-19T17_43_08Z.json
2025-01-30 09:04:59+00:00: connect-audio-files/to_process_transcripts/4d8deee2-8062-40e8-b580-d45d79e94abe_analysis_2025-01-15T22_57_07Z.json
2025-01-30 09:04:56+00:00: connect-audio-files/to_proces

# Validations

In [20]:
df_inter_calls_data = df_inter_calls_data.drop(['CALL_JOB_STATUS'], axis=1)

In [21]:
df_inter_calls_data.head()

,CONTACT_ID,CALL_TEXT,CALL_SUMMARY,TOPIC,CATEGORY,CATEGORY_GENERATED,SUB_CATEGORY,SUB_CATEGORY_GENERATED,AGENT_COACHING,AGENT_SPEECH_SPEED,CUSTOMER_SPEECH_SPEED,TOTAL_TALKTIME_AGENT_SECOND,TOTAL_TALKTIME_CUSTOMER_SECOND,TOTAL_TALKTIME_CALL_SECOND,TOTAL_DURATION_CALL_SECOND,TOTAL_DEAD_AIR_CALL_SECOND,CALL_LANGUAGE,CALL_S3_URI,AC_LAST_MODIFIED_DATE,LOAD_DATE
0,309a0db8-2735-4537-988a-a66bff37c159,"Hi, I'm looking for Mr. {{NAME}}. Speaking. Hi...",The call involved transferring a solar lease t...,Solar Lease Transfer,Static Category TBD,Customer Service,Static Sub-Category TBD,Account Management,strengths:\n - Clearly explained complex info...,221,155,627258,321628,948886,1044130,95244,en-US,s3://amazon-connect-39f6aa5d9242/connect/posig...,2025-01-17 21:06:34,2025-02-11 17:33:41.220487
1,3c2ddea2-d6e3-4acd-8abd-6eb98c192bd8,Thank you for calling {{ORGANIZATION}}. This i...,Customer called to make a payment of $64.99. ...,Payment Processing,Static Category TBD,Finance,Static Sub-Category TBD,Payment Collection,strengths:\n - Successfully processed payment...,160,121,164142,240297,404439,475679,71240,en-US,s3://amazon-connect-39f6aa5d9242/connect/posig...,2025-01-06 18:29:29,2025-02-11 17:34:01.648864
2,3e747be8-b57e-4e90-954d-dabc1e9a623a,"Hello, Ms. {{NAME}}, uh, for some reason we wa...",Customer called regarding inaccurate informati...,Sales Misinformation and Complaint,Static Category TBD,Customer Service,Static Sub-Category TBD,Sales Issue,strengths:\n - Empathetic listening and under...,167,192,407773,242444,650217,665429,15212,en-US,s3://amazon-connect-39f6aa5d9242/connect/posig...,2025-01-14 19:30:16,2025-02-11 17:35:11.514886
3,4a0a35d3-6221-4ccd-be93-83938c24a544,Hello? Good afternoon. My name is {{NAME}}. I'...,A customer called regarding a lease transfer. ...,Lease Transfer,Static Category TBD,Real Estate,Static Sub-Category TBD,Tenant Services,strengths:\n - Successfully clarified custome...,184,179,68808,38579,107387,118739,11352,en-US,s3://amazon-connect-39f6aa5d9242/connect/posig...,2025-01-17 19:04:29,2025-02-11 17:35:23.402853
4,4abe04de-5d7d-4eb9-a999-99d1c6ef54c1,"Good, good morning. This is {{NAME}} with {{OR...",Customer reported high electricity bills despi...,Billing Discrepancy/Solar System Malfunction,Static Category TBD,Customer Service,Static Sub-Category TBD,Technical Issue/Billing Inquiry,strengths:\n - Patiently explained complex bi...,144,178,486982,340959,827941,1423500,595559,en-US,s3://amazon-connect-39f6aa5d9242/connect/posig...,2024-12-19 17:43:08,2025-02-11 17:35:54.439372


In [22]:
print(f"Writing Dataframe to Snowflake.")
conn = sc.connect(**conn_params)
table_name ='SRC_GCP_INTER_CALLS'
insert_new_records(conn, table_name, df_inter_calls_data)

table_name ='SRC_GCP_INTRA_CALLS'
success, nchunks, nrows, _ = write_pandas(conn, df_intra_calls_data, 'SRC_GCP_INTRA_CALLS', auto_create_table=True)
conn.close()

Writing Dataframe to Snowflake.
Inserted 18 new records
Skipped 0 existing records
